In [ ]:
import pandas as pd

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from typing import List

import model_weightinit

degree_sign = u'\N{DEGREE SIGN}'

In [ ]:
ft_percents = [0.0, 0.2, 0.4, 0.6, 0.8]

In [ ]:
def plot_errors_and_trend(ft_percents: List[float], from_building: str, from_tower: int, to_building: str, season: str = None):

    # MAKE PREDICTIONS AND PLOT TRENDS
    rmse_list = []
    mabs_list = []

    fig_trend = make_subplots(rows=5, cols=2, subplot_titles=(f"{from_building} 1 to {to_building} 1 finetuned 0%", f"{from_building} 1 to {to_building} 2 finetuned 0%", f"{from_building} 1 to {to_building} 1 finetuned 20%", f"{from_building} 1 to {to_building} 2 finetuned 20%", f"{from_building} 1 to {to_building} 1 finetuned 40%", f"{from_building} 1 to {to_building} 2 finetuned 40%", f"{from_building} 1 to {to_building} 1 finetuned 60%", f"{from_building} 1 to {to_building} 2 finetuned 60%", f"{from_building} 1 to {to_building} 1 finetuned 80%", f"{from_building} 1 to {to_building} 2 finetuned 80%"))

    for tow in range(1,3):
        to_features = [f'{to_building}_Tower_{tow} enteringWaterTemp', f'{to_building}_Tower_{tow} outdoorAirDryBulb', f'{to_building}_Tower_{tow} outdoorAirWetBulb', f'{to_building}_Tower_{tow} vfdPercent', f'{to_building}_Tower_{tow} fanA_vfdPower', f'{to_building}_Tower_{tow} fanB_vfdPower', f'{to_building}_Tower_{tow} dayOfWeek', f'{to_building}_Tower_{tow} hourOfDay', f'{to_building}_Tower_{tow} efficiency']
        to_target = f'{to_building}_Tower_{tow} leavingWaterTemp'

        figrow, figcol = 1, tow

        rmse_list.append([])
        mabs_list.append([])

        for ft_percent in ft_percents:
            rmse, fig, mabs = model_weightinit.weight_init_transfer(from_building_name=from_building, from_tower_number=from_tower, to_building_name=to_building, to_tower_number=tow, to_features=to_features, to_target=to_target, to_season=season, finetuning_percentage=ft_percent, display_results=True, use_delta=True )
            rmse_list[tow-1].append(rmse)
            mabs_list[tow-1].append(mabs)
            
            fig_trend.add_trace(fig.data[0], row=figrow, col=figcol)
            fig_trend.add_trace(fig.data[1], row=figrow, col=figcol)
            fig_trend.update_xaxes(title_text="time", row=figrow, col=figcol)
            fig_trend.update_yaxes(title_text=f"{to_building} {tow} Leaving Water Temperature ({degree_sign}F)", row=figrow, col=figcol)

            figrow += 1

    fig_trend.update_layout(width=1000, height=1500)
    fig_trend.show()

    if not season: season = "all_year"
    fig_trend.write_html(f"../plots/weight_initialized/interbuilding_transfers/{from_building}{from_tower}_to_{to_building}_trend_{season}.html")

    # PLOT ERROR GRADIENT

    # tower 1: create a DataFrame and line graph using Plotly Express
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 'RMSE': rmse_list[0], 'MABS': mabs_list[0]})
    fig1 = px.line(print_df, y=['RMSE', 'MABS'], x="finetuning data percentage", title='Errors')
    # tower 2: create a DataFrame and line graph using Plotly Expres
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 'RMSE': rmse_list[1], 'MABS': mabs_list[1]})
    fig2 = px.line(print_df, y=['RMSE', 'MABS'], x="finetuning data percentage", title='Errors')

    figfinal = make_subplots(rows=1, cols=2)
    figfinal.add_trace(fig1.data[0], row=1, col=1)
    figfinal.add_trace(fig1.data[1], row=1, col=1)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=1)
    figfinal.update_yaxes(title_text="Error", row=1, col=1)

    figfinal.add_trace(fig2.data[0], row=1, col=2)
    figfinal.add_trace(fig2.data[1], row=1, col=2)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=2)
    figfinal.update_yaxes(title_text="Error", row=1, col=2)

    figfinal.update_layout(title_text=f"{from_building} Tower {from_tower} to {to_building} Tower 1, {from_building} Tower {from_tower} to {to_building} Tower 2 ({season}) (feature extraction)", showlegend=True)

    figfinal.show()
    figfinal.write_html(f"../plots/weight_initialized/interbuilding_transfers/{from_building}{from_tower}_to_{to_building}_gradient_{season}.html")

    # PLOT PERFORMANCE IMPROVEMENT RATIO

    pir = [[(tower_rmses[0] - rmse) / tower_rmses[0] for rmse in tower_rmses] for tower_rmses in rmse_list]

    # tower 1: create a DataFrame and line graph using Plotly Express
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 'Performance improvement ratio': pir[0]})
    fig1 = px.line(print_df, y=["Performance improvement ratio"], x="finetuning data percentage", title="Performance improvement ratio")
    # tower 2: create a DataFrame and line graph using Plotly Expres
    print_df = pd.DataFrame({'finetuning data percentage': [fp*100 for fp in ft_percents], 'Performance improvement ratio': pir[1]})
    fig2 = px.line(print_df, y=["Performance improvement ratio"], x="finetuning data percentage", title="Performance improvement ratio")
    
    figfinal = make_subplots(rows=1, cols=2)
    figfinal.add_trace(fig1.data[0], row=1, col=1)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=1)
    figfinal.update_yaxes(title_text="Performance improvement ratio", row=1, col=1)

    figfinal.add_trace(fig2.data[0], row=1, col=2)
    figfinal.update_xaxes(title_text="Finetuning data percentage", row=1, col=2)
    figfinal.update_yaxes(title_text="Performance improvement ratio", row=1, col=2)

    figfinal.update_layout(title_text=f"{from_building} Tower {from_tower} to {to_building} Tower 1, {from_building} Tower {from_tower} to {to_building} Tower 2 ({season}) (feature extraction)", showlegend=True)

    figfinal.show()
    figfinal.write_html(f"../plots/weight_initialized/interbuilding_transfers/{from_building}{from_tower}_to_{to_building}_pir_{season}.html")

## ESB 1 to Kissam 1 & 2

In [ ]:
plot_errors_and_trend(ft_percents=ft_percents, from_building="ESB", from_tower=1, to_building="Kissam", season="summer")

## ESB 2 to Kissam 1 & 2

In [ ]:
plot_errors_and_trend(ft_percents=ft_percents, from_building="ESB", from_tower=2, to_building="Kissam", season="summer")

## Kissam 1 to ESB 1 & 2

In [ ]:
plot_errors_and_trend(ft_percents=ft_percents, from_building="Kissam", from_tower=1, to_building="ESB", season="summer")

## Kissam 2 to ESB 1 & 2

In [ ]:
plot_errors_and_trend(ft_percents=ft_percents, from_building="Kissam", from_tower=2, to_building="ESB", season="summer")

# --- Testing transfers on MRB ----

## ESB 1 to MRB 1 & 2

In [ ]:
plot_errors_and_trend(ft_percents=ft_percents, from_building="ESB", from_tower=1, to_building="MRB", season="summer")

## Kissam 1 to MRB 1 & 2

In [ ]:
plot_errors_and_trend(ft_percents=ft_percents, from_building="Kissam", from_tower=1, to_building="MRB", season="summer")